In [21]:
from  langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import TextLoader
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM

test = TextLoader("notes.txt")
docs = test.load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size =100, chunk_overlap = 0)
chunks = text_splitter.split_documents(docs)
embeded = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-V2")
vectorstore = FAISS.from_documents(chunks, embeded)
retriever = vectorstore.as_retriever()
model_id = "sshleifer/tiny-gpt2"  # Use a small model if running locally
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Create Hugging Face pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    temperature=0.7,
    do_sample=True,
)

# Use in LangChain
llm = HuggingFacePipeline(pipeline=pipe)
qa = RetrievalQA.from_chain_type(llm=llm, retriever = retriever)
def self_rag_query(Question):
    print("first attempt without retriever:")
    first_answer = llm.predict(f"Q:{Question}\nA:")
    if "Iam not sure"in first_answer or len(first_answer) <30:
        print("low confidence Retrieving context and trying again...")
        improved_answer = qa.run(Question)
        return improved_answer
    else:
        return first_answer
response = self_rag_query("What is Television Mini?")
print("\nFinal answer:", response)
 



Created a chunk of size 174, which is longer than the specified 100
Created a chunk of size 169, which is longer than the specified 100
Created a chunk of size 647, which is longer than the specified 100
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


first attempt without retriever:

Final answer: Q:What is Television Mini?
A: circumcisediken Observ Hancock reviewing hauled ESVdit ESVScene subst circumcisedtingSherScene Money Observ Probiken Observ dispatch stairs Money Motorola credibility conservation directly Amphreement credibility hauledimura directlyJD Brewmediately vendors trilogy Brew Money


Created a chunk of size 174, which is longer than the specified 100
Created a chunk of size 169, which is longer than the specified 100
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


first attempt without retriever:

Final answer: Q:What is the API Key Management?
A:004 Motorola circumcised directly ESV subst autonomyJD stairs Observ TA Amph stairsohopresspress Prob credibility reviewingSher TA vendors ESV Amph Motorolahibithibit Observ rebornmediately TA MoneyreementohoScene TA heir Money
